In [1]:
import math
from datasets import load_dataset
from transformers import Trainer, TrainingArguments

from src.llm_trainer import LLMTrainer

/usr/local/lib/python3.10/dist-packages/transformers/utils/hub.py:124: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


## Define the SAME formatting you used for training

## Load base model, then load LoRA adapters

In [2]:
print(type(trainer.model))
print(hasattr(trainer.model, "_orig_mod"))  # True usually means torch.compile wrapper


NameError: name 'trainer' is not defined

In [2]:
# ---- 2) Load base model, then load LoRA adapters ----
BASE_MODEL = "Qwen/Qwen2.5-3B"
LORA_PATH = "../outputs/checkpoints/lora_qwen/"      # folder you saved adapters to
TEST_JSON = "../data/processed/test.json"

trainer = LLMTrainer(
    model_name=BASE_MODEL,
    model_type="decoder",
    load_in_4bit=False
)

trainer.configure_lora(r=8, lora_alpha=32)
trainer.load_lora_adapters(LORA_PATH)

metrics = trainer.evaluate_classification(
    test_path=TEST_JSON,
    labels=["negative", "neutral", "positive"],
    average="macro",
    max_new_tokens=6
)

print(metrics["accuracy"], metrics["precision"], metrics["recall"], metrics["f1"], metrics["auc_ovr"])
print(metrics["confusion_matrix"])
print(metrics["classification_report"])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

trainable params: 1,843,200 || all params: 3,087,781,888 || trainable%: 0.059693335438076124


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_toke

0.8264462809917356 0.7883524413112108 0.8283463143711591 0.8003751158113573 0.9404823493298138
[[ 21   1   1]
 [  9 129   9]
 [  1  21  50]]
              precision    recall  f1-score   support

    negative       0.68      0.91      0.78        23
     neutral       0.85      0.88      0.87       147
    positive       0.83      0.69      0.76        72

    accuracy                           0.83       242
   macro avg       0.79      0.83      0.80       242
weighted avg       0.83      0.83      0.83       242



## Load + preprocess test.json

In [ ]:
ds = load_dataset("json", data_files={"test": TEST_JSON})["test"]

tokenizer = trainer.tokenizer
ds = ds.map(lambda ex: format_prompt(ex, tokenizer.eos_token))

def tokenize(batch):
    return tokenizer(
        batch["text"],
        truncation=True,
        max_length=128,
        padding=False,   # dynamic padding later
    )

tokenized_ds = ds.map(
    tokenize,
    batched=True,
    batch_size=128,
    remove_columns=ds.column_names,
    desc="Tokenizing",
)

## Evaluate with HF Trainer

In [6]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,   # causal LM
)

In [7]:
# ---- 4) Evaluate with HF Trainer ----
eval_args = TrainingArguments(
    output_dir="../outeval_out",
    per_device_eval_batch_size=4,
    dataloader_drop_last=False,
    report_to=[],  # avoid wandb errors
)

In [8]:
eval_trainer = Trainer(
    model=trainer.model,
    args=eval_args,
    eval_dataset=tokenized_ds,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

In [9]:
metrics = eval_trainer.evaluate()

In [10]:
metrics


{'eval_loss': 3.3138420581817627,
 'eval_runtime': 943.8519,
 'eval_samples_per_second': 0.256,
 'eval_steps_per_second': 0.065}